### Connection

In [1]:
import hopsworks
import numpy as np
import pandas as pd
import datetime

# project = hopsworks.login()
# fs = project.get_feature_store()

project = hopsworks.login(host="57.129.23.31",
                          port=28181,
                          project="fmon2",
                          api_key_value="3Ypfk2ejeILxvJXc.jW9Kc3wHTDOcegxDILOc7GAK4t4vse2wo1WpPxdhMAPO8ZhibQCxRUvIND5MH15L")
fs = project.get_feature_store()

2025-04-21 08:19:30,632 INFO: Initializing external client
2025-04-21 08:19:30,633 INFO: Base URL: https://57.129.23.31:28181


ConnectTimeout: HTTPSConnectionPool(host='57.129.23.31', port=28181): Max retries exceeded with url: /hopsworks-api/api/variables/versions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1ce739c280>, 'Connection to 57.129.23.31 timed out. (connect timeout=None)'))

In [ ]:
trans_fg = fs.get_or_create_feature_group(name="trans_fg", version=1)

In [ ]:
fmcs = trans_fg.get_feature_monitoring_configs()

In [ ]:
# for fmc in fmcs:
#     print("Deleting FMC: " + fmc.name)
#     fmc.delete() 

### FM meta objects

#### FM config, window configs, statistics config, comparison config

In [ ]:
fmc = trans_fg.get_feature_monitoring_configs(name="fg_some_ref_spec_monitoring_reference_sliding")

In [ ]:
def print_fmc(fmc):
    print(fmc.__repr__())
    print(fmc.id)
    print(fmc.name)
    print(fmc.description)
    print(fmc.job_name)
    print(fmc.feature_monitoring_type)
    print(fmc.job_schedule.__repr__())

print_fmc(fmc)


In [ ]:
def print_window_config(wc):
    if wc is None: return
    print(wc.__repr__())
    print(wc.id)
    print(wc.window_config_type)
    print(wc.window_length)
    print(wc.time_offset)
    print(wc.training_dataset_version)
    print(wc.row_percentage)

In [ ]:
print_window_config(fmc.detection_window_config)

In [ ]:
print_window_config(fmc.reference_window_config)

In [ ]:
def print_fsc(fsc):
    print(fsc.__repr__())
    print(fsc.id)
    print(fsc.feature_name)
    print(fsc.statistics_comparison_configs)

print_fsc(fmc.feature_statistics_configs[0])

In [ ]:
def print_scc(scc):
    print(scc.__repr__())
    print(scc.id)
    print(scc.metric)
    print(scc.threshold)
    print(scc.relative)
    print(scc.strict)
    print(scc.specific_value)

print_scc(fmc.feature_statistics_configs[0].statistics_comparison_configs[0])

#### FM results, statistics results, statistics comparison

In [ ]:
fmrs = fmc.get_history()
fmrs

In [ ]:
def print_fmr(fmr):
    print(fmr.__repr__())
    print(fmr.id)
    print(fmr.feature_store_id)
    print(fmr.feature_monitoring_config_id)
    print(fmr.execution_id)
    print(fmr.monitoring_time)
    print(fmr.empty_detection_window)
    print(fmr.empty_reference_window)
    print(fmr.shifted_feature_names)
    print(fmr.shift_detected)
    print(fmr.feature_statistics_results)

print_fmr(fmrs[0])

In [ ]:
def print_fsr(fsr):
    print(fsr.__repr__())
    print(fsr.id)
    print(fsr.feature_name)
    print(fsr.detection_statistics_id)
    print(fsr.reference_statistics_id)
    print(fsr.shifted_metric_names)
    print(fsr.shift_detected)
    print(fsr.detection_statistics)
    print(fsr.reference_statistics)
    print(fsr.statistics_comparison_results)

print_fsr(fmrs[0].feature_statistics_results[0])

# TODO: Fetch statistics values!!!

In [ ]:
def print_scr(scr):
    print(scr.__repr__())
    print(scr.id)
    print(scr.feature_statistics_result_id)
    print(scr.statistics_comparison_config_id)
    print(scr.shift_detected)
    print(scr.difference)

print_scr(fmrs[0].feature_statistics_results[0].statistics_comparison_results[0])

### FM with Detection only

---

#### All features (default window)

In [ ]:
fg_all_features_monitoring = trans_fg.create_scheduled_statistics(
    name="fg_all_features_monitoring",
    description="Compute statistics on all data of all features of the Feature Group on a daily basis",
    cron_expression="0 0,20,40 * ? * * *",
).save()

In [ ]:
fg_all_features_monitoring.disable()

fg_all_features_monitoring.run_once()

In [ ]:
fg_all_features_monitoring = trans_fg.get_feature_monitoring_configs(name="fg_all_features_monitoring")
# fg_all_features_monitoring.delete()

#### Subset of features (default window)

In [ ]:
fg_some_features_monitoring = trans_fg.create_scheduled_statistics(
    name="fg_some_features_monitoring",
    description="Compute statistics on all data of some features of the Feature Group on a daily basis",
    feature_names=["amount", "country"],
    cron_expression="0 2,22,42 * ? * * *",
).save()

In [ ]:
fg_some_features_monitoring.disable()

fg_some_features_monitoring.run_once()

In [ ]:
fg_some_features_monitoring = trans_fg.get_feature_monitoring_configs(name="fg_some_features_monitoring")
# fg_some_features_monitoring.delete()

#### Subset of features (custom window)

In [ ]:
fg_det_window_some_features_monitoring = trans_fg.create_scheduled_statistics(
    name="fg_det_window_some_features_monitoring",
    description="Compute statistics on windowed data of some features of the Feature Group on a daily basis",
    feature_names=["amount", "country"],
    cron_expression="0 4,24,44 * ? * * *",
).with_detection_window(
    time_offset="1w", # fetch data from the last week
    row_percentage=0.8,
).save()

In [ ]:
fg_det_window_some_features_monitoring.disable()

fg_det_window_some_features_monitoring.run_once()

In [ ]:
fg_det_window_some_features_monitoring = trans_fg.get_feature_monitoring_configs(name="fg_det_window_some_features_monitoring")
# fg_det_window_some_features_monitoring.delete()

### FM with References

---

#### Single feature (specific values)

In [ ]:
fg_amount_monitoring_reference_value = trans_fg.create_feature_monitoring(
    name="fg_amount_monitoring_reference_value",
    cron_expression="0 6,26,46 * ? * * *",
    description="Compute descriptive statistics on amount Feature in the last week of data inserted in the Feature Group",
).with_detection_window(
    time_offset="1w", # fetch data from the last week
    row_percentage=0.8,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=50,
    specific_value=100,
#     strict=False,
#     relative=True,
).compare_on(
    feature_name="amount",
    metric="sum",
    threshold=0.6,
    relative=True,
    specific_value=400,
#     strict=False,
#     relative=True,
).save()

In [ ]:
# fg_amount_monitoring_reference_value.disable()

fg_amount_monitoring_reference_value.run_once()


In [ ]:
fg_amount_monitoring_reference_value = trans_fg.get_feature_monitoring_configs(name="fg_amount_monitoring_reference_value")
# fg_amount_monitoring_reference_value.delete()

#### Subset of features (reference window)

In [ ]:
fg_some_monitoring_reference_sliding = trans_fg.create_feature_monitoring(
    name="fg_some_monitoring_reference_sliding",
    cron_expression="0 8,28,48 * ? * * *",
    description="Compute and compare descriptive statistics on the some Feature on a daily basis to the same statistics computed in the previous week",
).with_detection_window(
    time_offset="3h", # fetch data from inserted throughout the last day
    row_percentage=0.8,
).with_reference_window(
    time_offset="1d3h", # fetch data from the start of same day of the previous week
    window_length="1d", # limit the reference window to the same day of the previous week
    row_percentage=0.8,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=0.1,
    relative=True,
    specific_value=10,
).compare_on(
    feature_name="amount",
    metric="std_dev",
    threshold=0.1,
    relative=True,
).compare_on(
    feature_name="amount",
    metric="sum",
    threshold=0.4, # allow for a 40% difference between the two windows before triggering an alert
    relative=True,
).compare_on(
    feature_name="fee",
    metric="min",
    threshold=0.1,
    relative=False,
).compare_on(
    feature_name="fee",
    metric="max",
    threshold=3,
    relative=False,
).save()

In [ ]:
fg_some_monitoring_reference_sliding.disable()

fg_some_monitoring_reference_sliding.run_once()

In [ ]:
fg_some_monitoring_reference_sliding = trans_fg.get_feature_monitoring_configs(name="fg_amount_monitoring_reference_value")
# fg_some_monitoring_reference_sliding.delete()

In [ ]:
# Fetch results
monitoring_results = fg_some_monitoring_reference_sliding.get_history(
    start_time=datetime.datetime.now() - datetime.timedelta(days=1), # fetched data inserted in the last day
    end_time=datetime.datetime.now(),
)
print(monitoring_results[0])

#### Subset of features (reference window and specific values)

In [ ]:
fg_some_ref_spec_monitoring_reference_sliding = trans_fg.create_feature_monitoring(
    name="fg_some_ref_spec_monitoring_reference_sliding",
        cron_expression="0 10,30,50 * ? * * *",
    description="Compute and compare descriptive statistics on the some Feature on a daily basis to the same statistics computed in the previous week",
).with_detection_window(
    time_offset="3h", # fetch data from inserted throughout the last day
    row_percentage=0.8,
).with_reference_window(
    time_offset="1d3h", # fetch data from the start of same day of the previous week
    window_length="1d", # limit the reference window to the same day of the previous week
    row_percentage=0.8,
).compare_on(
    feature_name="amount",
    metric="mean",
    threshold=50,
    specific_value=100,
).compare_on(
    feature_name="amount",
    metric="std_dev",
    threshold=0.1,
    relative=True,
).compare_on(
    feature_name="amount",
    metric="sum",
    threshold=0.6,  # allow for a 60% difference between the two windows before triggering an alert
    relative=True,
    specific_value=400,
).compare_on(
    feature_name="fee",
    metric="min",
    threshold=0.1,
    relative=False,
).compare_on(
    feature_name="fee",
    metric="max",
    threshold=3,
    relative=False,
).save()

In [ ]:
fg_some_ref_spec_monitoring_reference_sliding.disable()

fg_some_ref_spec_monitoring_reference_sliding.run_once()

In [ ]:
fg_some_ref_spec_monitoring_reference_sliding = trans_fg.get_feature_monitoring_configs(name="fg_some_ref_spec_monitoring_reference_sliding")
fg_some_ref_spec_monitoring_reference_sliding.delete()